In [ ]:
// upload the kagge api key
from google.colab import files
files.upload()


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install -q kaggle


In [ ]:
!kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset
!unzip real-life-violence-situations-dataset.zip -d violence_dataset


Dataset URL: https://www.kaggle.com/datasets/mohamedmustafa/real-life-violence-situations-dataset
License(s): copyright-authors
Archive:  real-life-violence-situations-dataset.zip
  inflating: violence_dataset/Real Life Violence Dataset/NonViolence/NV_1.mp4  
  inflating: violence_dataset/Real Life Violence Dataset/NonViolence/NV_10.mp4  
  inflating: violence_dataset/Real Life Violence Dataset/NonViolence/NV_100.mp4  
  inflating: violence_dataset/Real Life Violence Dataset/NonViolence/NV_1000.mp4  
  inflating: violence_dataset/Real Life Violence Dataset/NonViolence/NV_101.mp4  
  inflating: violence_dataset/Real Life Violence Dataset/NonViolence/NV_102.mp4  
  inflating: violence_dataset/Real Life Violence Dataset/NonViolence/NV_103.mp4  
  inflating: violence_dataset/Real Life Violence Dataset/NonViolence/NV_104.mp4  
  inflating: violence_dataset/Real Life Violence Dataset/NonViolence/NV_105.mp4  
  inflating: violence_dataset/Real Life Violence Dataset/NonViolence/NV_106.mp4  
  

In [ ]:
!pip install imageio opencv-python-headless


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

IMG_SIZE = 112
FRAMES_PER_CLIP = 16

def extract_video_clips(video_path, max_clips=5):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frames.append(frame)

    cap.release()

    # Make clips
    clips = []
    for i in range(0, len(frames) - FRAMES_PER_CLIP, FRAMES_PER_CLIP):
        clip = frames[i:i + FRAMES_PER_CLIP]
        if len(clip) == FRAMES_PER_CLIP:
            clips.append(np.array(clip))
        if len(clips) >= max_clips:
            break

    return clips


In [ ]:
!pip install tensorflow opencv-python-headless


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, SpatialDropout3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Hyperparameters
IMG_SIZE = 112  # Image size for each frame
FRAMES_PER_CLIP = 32  # Number of frames per video clip (increase context)
BATCH_SIZE = 8  # Batch size
EPOCHS = 15  # Increase epochs to allow for better training

# Paths
violence_dir = '/content/violence_dataset/Real Life Violence Dataset/Violence'
nonviolence_dir = '/content/violence_dataset/Real Life Violence Dataset/NonViolence'

video_list = []

# Load videos and their labels
for video_file in os.listdir(violence_dir):
    video_path = os.path.join(violence_dir, video_file)
    video_list.append((video_path, 1))  # Label 1 for violence

for video_file in os.listdir(nonviolence_dir):
    video_path = os.path.join(nonviolence_dir, video_file)
    video_list.append((video_path, 0))  # Label 0 for non-violence

# Shuffle and split
random.shuffle(video_list)
split = int(0.8 * len(video_list))
train_videos = video_list[:split]
val_videos = video_list[split:]

# Function to extract clips
def extract_video_clips(video_path, max_clips=5):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frames.append(frame)

    cap.release()

    # Make clips
    clips = []
    for i in range(0, len(frames) - FRAMES_PER_CLIP, FRAMES_PER_CLIP):
        clip = frames[i:i + FRAMES_PER_CLIP]
        if len(clip) == FRAMES_PER_CLIP:
            clips.append(np.array(clip))
        if len(clips) >= max_clips:
            break

    return clips

# Data generator with augmentation
def data_generator(video_list, batch_size=BATCH_SIZE):
    while True:
        random.shuffle(video_list)
        batch_X, batch_y = [], []

        for path, label in video_list:
            clips = extract_video_clips(path)
            for clip in clips:
                batch_X.append(clip)
                batch_y.append(label)

                if len(batch_X) == batch_size:
                    # Normalize and apply augmentation
                    batch_X = np.array(batch_X) / 255.0
                    batch_y = np.array(batch_y)
                    yield batch_X, batch_y
                    batch_X, batch_y = [], []

# 3D CNN model
def create_3d_cnn(input_shape=(32, 112, 112, 3)):
    model = Sequential()

    # Add 3D convolution layers
    model.add(Conv3D(64, (3, 3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(SpatialDropout3D(0.2))  # Prevent overfitting by adding spatial dropout

    model.add(Conv3D(128, (3, 3, 3), activation='relu', padding='same'))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(SpatialDropout3D(0.3))

    model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same'))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(SpatialDropout3D(0.3))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model


# Create the model
model = create_3d_cnn()

# Callbacks for better training
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)

# Train the model
train_gen = data_generator(train_videos, batch_size=BATCH_SIZE)
val_gen = data_generator(val_videos, batch_size=BATCH_SIZE)

model.fit(
    train_gen,
    steps_per_epoch=len(train_videos) // BATCH_SIZE,
    validation_data=val_gen,
    validation_steps=len(val_videos) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 214s 964ms/step - accuracy: 0.6429 - loss: 4.1715 - val_accuracy: 0.4050 - val_loss: 3.3338 - learning_rate: 1.0000e-04
Epoch 2/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 147s 736ms/step - accuracy: 0.6913 - loss: 0.7145 - val_accuracy: 0.4950 - val_loss: 1.4142 - learning_rate: 1.0000e-04
Epoch 3/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 173s 868ms/step - accuracy: 0.7019 - loss: 0.5786 - val_accuracy: 0.6500 - val_loss: 0.5923 - learning_rate: 1.0000e-04
Epoch 4/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 149s 745ms/step - accuracy: 0.7096 - loss: 0.6102 - val_accuracy: 0.7925 - val_loss: 0.4553 - learning_rate: 1.0000e-04
Epoch 5/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 144s 722ms/step - accuracy: 0.7821 - loss: 0.5015 - val_accuracy: 0.8300 - val_loss: 0.3873 - learning_rate: 1.0000e-04
Epoch 6/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 196s 983ms/step - accuracy: 0.7793 - loss: 0.5168 - val_accuracy: 0.7525 - val_loss: 0.6049 - learning_rate: 1.0000e-04
Epoch 7/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Specify the path to save the model on Google Drive
save_path = '/content/drive/MyDrive/violence_detection_model_epoch8.h5'

# Save the model
model.save(save_path)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Specify the path to save the model on Google Drive
save_path = '/content/drive/MyDrive/violence_detection_model_epoch10.h5'

# Save the model
model.save(save_path)


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('/content/drive/MyDrive/violence_detection_model_epoch10.h5')

# Load the video
video_path = '/content/nonviolentvideo.mp4'  # Video path updated
cap = cv2.VideoCapture(video_path)

# Model input size and clip parameters
IMG_SIZE = 112
FRAMES_PER_CLIP = 32  # Adjusted frames per clip to match the model's input
frame_count = 0
violence_threshold = 0.7  # Adjust if needed
violence_detected = False

# Function to extract frames as clips
def extract_frames(video_path, max_clips=5):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frames.append(frame)

    cap.release()

    # Make clips
    clips = []
    for i in range(0, len(frames) - FRAMES_PER_CLIP, FRAMES_PER_CLIP):
        clip = frames[i:i + FRAMES_PER_CLIP]
        if len(clip) == FRAMES_PER_CLIP:
            clips.append(np.array(clip))
        if len(clips) >= max_clips:
            break

    return clips

# Extract frames from the video
video_clips = extract_frames(video_path)

# Prediction loop for video clips
for clip_index, clip in enumerate(video_clips):
    clip = np.expand_dims(clip, axis=0)  # Add batch dimension
    clip_normalized = clip / 255.0  # Normalize the clip

    # Check the input shape to match model's expectation
    print(f"Clip shape before prediction: {clip_normalized.shape}")  # Should print (1, 32, 112, 112, 3)

    # Prediction
    prediction = model.predict(clip_normalized)[0][0]

    if prediction > violence_threshold:
        print(f"[ALERT] Clip {clip_index+1} - Violence Detected with confidence: {prediction:.2f}")
        violence_detected = True
    else:
        print(f"[OK] Clip {clip_index+1} - No Violence, confidence: {prediction:.2f}")

# Final verdict
if violence_detected:
    print("🚨 Final Verdict: Violence detected in the video.")
else:
    print("✅ Final Verdict: No violence detected in the video.")


Clip shape before prediction: (1, 32, 112, 112, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 650ms/step
[OK] Clip 1 - No Violence, confidence: 0.04
Clip shape before prediction: (1, 32, 112, 112, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
[OK] Clip 2 - No Violence, confidence: 0.16
Clip shape before prediction: (1, 32, 112, 112, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
[OK] Clip 3 - No Violence, confidence: 0.16
Clip shape before prediction: (1, 32, 112, 112, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
[OK] Clip 4 - No Violence, confidence: 0.43
Clip shape before prediction: (1, 32, 112, 112, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
[OK] Clip 5 - No Violence, confidence: 0.66
✅ Final Verdict: No violence detected in the video.
